In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#minha funcao para mostrar as imagens
def imshow(im,big=False):
    if big:
        plt.figure(figsize=(20,20))
    else:
        plt.figure()
    if len(im.shape)<3:
        plt.imshow(im, cmap='gray')#As cores ficam zoadas no Jupyter se deixar sem esse parâmetro
    else:
        plt.imshow(im) 
    plt.axis('off')
    plt.show()

In [ ]:
def f2i(im):#float to int
    return np.round(im * 255.0).astype('uint8')
def i2f(im):#int to float
    return im.astype('float') / 255.0
def norm(im):
    return (im-im.min())/(im.max()-im.min())

### 1. Abrindo Imagem

60.bmp<br>
82.bmp<br>
114.bmp<br>
150.bmp<br>
205.bmp

In [ ]:
im = cv2.imread("205.bmp")
imshow(im)

### 2. Convertendo para escala de cinza

In [ ]:
def cinza(im):
    return norm(i2f(cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)))
im_gray = cinza(im)
imshow(im_gray)

### Gradiente

In [ ]:
def grad(im_gray,ksize=5):
    #sobel
    sobelx = cv2.Sobel(im_gray,cv2.CV_64F,1,0,ksize=ksize)
    sobely = cv2.Sobel(im_gray,cv2.CV_64F,0,1,ksize=ksize)
    im_grad = np.sqrt(sobelx**2+sobely**2)
    #normaliza
    im_grad = (im_grad-im_grad.min())/(im_grad.max()-im_grad.min())
    return im_grad
im_grad = grad(im_gray,7)
imshow(im_grad)

### Limiarização global

In [ ]:
def limiarizacao_global(im, limite = 0.22):
    ret,im_limiarizacao_global = cv2.threshold(im,limite, 1.0, cv2.THRESH_BINARY)
    return f2i(im_limiarizacao_global)
im_bin = limiarizacao_global(im_grad)
imshow(im_bin,True)

### Destroi pequenos objetos 

In [ ]:
def limpa(im,area_min = 50):
    ret, labels = cv2.connectedComponents(im)
    for i in range(ret):
        mask = labels==i
        if mask.sum()<area_min:
            im[mask] = 0
    return im
im_limp = limpa(im_bin)
imshow(im_limp,True)

### Morphology

In [ ]:
def close(im):
    kernel = np.array([[0,1,0],[1,1,1],[0,1,0]]).astype('uint8')
    #kernel = np.ones((3,3))
    #im_erode = cv2.erode(im_limiarizacao_global,kernel,iterations = 1)
    im_morp = cv2.morphologyEx(im, cv2.MORPH_CLOSE, kernel,iterations=1)
    #im_morp = cv2.morphologyEx(im, cv2.MORPH_OPEN, kernel,iterations=1)
    return im_morp
im_morp = close(im_limp)
imshow(im_morp,True)

### floodFill invertido

In [ ]:
def flood_fill_inv(im):
    im = im.copy()
    h, w = im.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    im_flood = cv2.floodFill(im.copy(),mask,(0,0),255)

    im[im_flood[1]<100] = 255
    return im
im_fill = flood_fill_inv(im_morp)
imshow(im_fill)

In [ ]:
def openn(im):
    kernel = np.array([[0,1,0],[1,1,1],[0,1,0]]).astype('uint8')
    im_morp = cv2.morphologyEx(im, cv2.MORPH_OPEN, kernel,iterations=1)
    return im_morp

im_open = openn(im_fill)
imshow(im_open)

### pinta labels
0 vermelho, 1 amarelo, 2 verde, 3 ciano, 4 azul, 5 rosa, 6 vermelho

In [ ]:
def pinta_labels(labels):
    # Map component labels to hue val
    label_hue = np.uint8((30*labels)%180)
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2RGB)

    # set bg label to black
    labeled_img[labels==0] = 0
    
    return labeled_img

# 1 Tentativa: Areas

In [ ]:
def labels(im):
    ret, labels = cv2.connectedComponents(im.astype('uint8'))
    return labels
im_labels = labels(im_open)
imshow(im_labels)

In [ ]:
def areas_blobs(im):
    qnt = im.max()
    #area de cada blob
    areas = np.zeros(qnt).astype(int)
    for i in range(1,qnt+1):
        areas[i-1] = np.sum(im==i)
    
    #areas = areas[areas>20]
    #estima a quantidade de arroz
    return areas

areas = areas_blobs(im_labels)
areas

In [ ]:
from scipy import stats
def moda(areas,i = 25,faz_moda2 = 0):
    #meio que moda, tenta adivinhar o tamanho medio de um arroz
    mode = stats.mode(np.round(areas/i)).mode[0]*i
    print('  moda1:',mode)
    estimativas = areas/mode
    estimativas_round =  np.round(estimativas).astype(int)
    
    return estimativas_round
estimativas = moda(areas)
estimativas,estimativas.sum()

In [ ]:
def pinta_qntd(im,estimativas):
    im2 = im.copy()
    i=1
    for e in estimativas:
        im2[im==i] = e 
        i+=1
    return pinta_labels(im2)
im_qtnd = pinta_qntd(im_labels,estimativas)
imshow(im_qtnd)

# Pipelines

In [ ]:
def pipeline1(path):
    
    im = cv2.imread(path)
    
    im_gray = cinza(im)
    im_grad = grad(im_gray,9)
    im_bin  = limiarizacao_global(im_grad,0.23)
    im_limp = limpa(im_bin)
    #im_close = close(im_limp)
    im_fill = flood_fill_inv(im_limp)
    im_open = openn(im_fill)
    im_labels = labels(im_open)

    areas = areas_blobs(im_labels)
    estimativas = moda(areas,25)
    im_qtnd = pinta_qntd(im_labels,estimativas)
    
    print('  estimativa',estimativas.sum())
    return im_qtnd

In [ ]:
def testa_pipeline(pipeline,paths):
    for path in paths:
        print(path)
        im = pipeline(path)
        #imshow(im,True)
        cv2.imwrite(path+'.png',im[:,:,::-1])

In [ ]:
%%time
paths = ["60.bmp","82.bmp","114.bmp","150.bmp","205.bmp"]
testa_pipeline(pipeline1,paths)